In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt




import warnings

from sklearn import linear_model

warnings.filterwarnings('ignore')

capture_site = pd.read_csv('data/CaptureSite_category.csv')

In [ ]:
capture_site

# Sample_sub formating

In [335]:
sample_sub = pd.read_csv('data/Sample_sub.csv')

In [ ]:
sample_sub.info()

In [336]:
def extract_capture_site(df):
    df['capture_site'] = df['ID'].apply(lambda x: x.split('_')[-2])
    return df

In [337]:
def extract_and_convert_week(df):
    # Extract the second to last element
    df['week_caught'] = df['ID'].apply(lambda x: x[-2:])

    # Convert to datetime with appropriate format for year and month ("%Y%m")
    df['week_caught'] = df['week_caught'].apply(lambda x : int(x))

    return df

In [338]:
def formating_sample_sub(df): 
    # extracting capture site id
    df = extract_capture_site(df)
    
    # extracting week of rascue
    df = extract_and_convert_week(df)
    
    # renaming columns to match training set
    df.rename(columns={'Capture_Number': 'turtles_rescued'}, inplace=True)
    
    # getting rid of mixed column
    df.drop(columns=['ID'], inplace=True)
    
    # Standartising prediction 
    df = df.groupby(['capture_site', 'week_caught'])['turtles_rescued'].sum().reset_index()
    
    return df

In [339]:
sample_sub = formating_sample_sub(sample_sub)

# Train_df formating

In [ ]:
train_df = pd.read_csv('data/train.csv')

In [ ]:
train_df.head()

In [ ]:
import pandas as pd
import re

# Standartising column names 
def standardize_column_names(col):
    # Replace spaces with underscores
    col = col.replace(' ', '_')
    # Insert underscore before each uppercase letter preceded by a lowercase letter or followed by a lowercase letter
    col = re.sub(r'(?<=[a-z])(?=[A-Z])', '_', col)
    col = re.sub(r'(?<=[A-Z])(?=[A-Z][a-z])', '_', col)
    # Convert to lower case
    col = col.lower()
    # Ensure single underscores only (in case of consecutive underscores from initial spaces)
    col = re.sub(r'_+', '_', col)
    return col

train_df.columns = [standardize_column_names(col) for col in train_df.columns]

# Printing the updated column names to verify the changes
print(train_df.columns)

In [ ]:
#train_df.groupby('ReleaseSite').size()

#We can drop the column Sex, since the most values are Unknown 
df = train_df.groupby('date_time_caught').size().reset_index(name='Size')
df.sort_values('Size', ascending= False).tail(60)


In [ ]:
def extract_number_split(s):
    num = s.split('_')[-1]
    return int(num)

extract_number_split('Fischer_5')

In [ ]:
train_df['fisher'] = train_df['fisher'].apply(extract_number_split)

train_df['fisher']

In [ ]:
train_df['researcher'] = train_df['researcher'].apply(extract_number_split)

train_df['researcher']

In [ ]:
train_df['capture_site'] = train_df['capture_site'].apply(extract_number_split)

train_df['capture_site']

In [ ]:

train_df['species'] = train_df['species'].apply(extract_number_split)

train_df['species']

In [ ]:

#train_df['ReleaseSite'] = train_df['ReleaseSite'].apply(extract_number_split)

#train_df['ReleaseSite']

In [ ]:
import pandas as pd

def convert_and_split_datetime(df, columns):
    """
    Convert specified datetime columns to timestamp and split into year and week columns
    with new names based on the original column names.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing the columns.
    columns (list): List of column names to convert and split.
    
    Returns:
    pd.DataFrame: The DataFrame with new year and week columns.
    """
    for column in columns:
        # Convert the column to datetime
        df[column] = pd.to_datetime(df[column], errors='coerce')

        # Extract the base name without 'date_time_' prefix
        base_name = column.replace('date_time_', '')

        # Create new columns for year and week with the desired names
        df[f'year_{base_name}'] = df[column].dt.year
        df[f'week_{base_name}'] = df[column].dt.isocalendar().week

        # Drop the original datetime column if desired
        df.drop(columns=[column], inplace=True)

    return df

# Example usage
# Assuming 'df' is your DataFrame
columns_to_convert = ['date_time_caught', 'date_time_release']
train_df = convert_and_split_datetime(train_df, columns_to_convert)

train_df.head()

In [ ]:
train_df.columns

In [ ]:
category_counts = train_df['week_caught'].value_counts()

# Plotting using seaborn
plt.figure(figsize=(14, 8))
sns.barplot(x=category_counts.index, y=category_counts.values, palette='viridis')
plt.xticks(rotation=90)
plt.xlabel('week')
plt.ylabel('Count')
plt.title('Distribution of Categories')
plt.show()

In [ ]:
category_counts = train_df['capture_site'].value_counts()

# Plotting using seaborn
plt.figure(figsize=(14, 8))
sns.barplot(x=category_counts.index, y=category_counts.values, palette='viridis')
plt.xticks(rotation=90)
plt.xlabel('capture_site')
plt.ylabel('Count')
plt.title('Distribution of Categories')
plt.show()

In [ ]:
train_df

In [ ]:
train_df.nunique()

In [ ]:
train_df.info()

In [ ]:
df = train_df.groupby('status').size().reset_index(name='size')
df.sort_values('size', ascending= False).head(60)

In [ ]:
#We want to start the 

train_df.query('Date_TimeRelease.isna()')

In [ ]:
train_df[['Lost_Tags', 'Tag_1', 'Tag_2','Rescue_ID' ,'Date_TimeCaught']].query('Tag_1.isna() and Tag_2.isna() and Lost_Tags.isna()')

In [ ]:
# Change the fisher to just a fisher ID, Same for Researcher

df = train_df.groupby('Researcher').size().reset_index(name='Size')
df.sort_values('Size', ascending= False).head(60)


In [ ]:
# Build in a function using apply to convert the date to Week calender

In [ ]:
#We can drop the column Sex, since the most values are Unknown 
df = train_df.groupby('Sex').size().reset_index(name='Size')
df.sort_values('Size', ascending= False).head(60)


# Baseline Model

In [317]:
baseline_df = train_df.groupby(['year_caught', 'capture_site', 'week_caught']).size().reset_index(name='turtles_rescued')
baseline_df

,year_caught,capture_site,week_caught,turtles_rescued
0,1998,11,28,1
1,1998,11,32,1
2,1998,11,39,2
3,1998,11,43,1
4,1998,11,45,1
...,...,...,...,...
7952,2018,27,36,1
7953,2018,27,38,1
7954,2018,27,45,1
7955,2018,28,44,1


In [318]:
baseline_df = baseline_df[~baseline_df['year_caught'].between(1988, 2006)].reset_index(drop=True)
baseline_df

,year_caught,capture_site,week_caught,turtles_rescued
0,2007,2,2,1
1,2007,2,3,2
2,2007,2,4,1
3,2007,2,5,1
4,2007,2,7,1
...,...,...,...,...
6461,2018,27,36,1
6462,2018,27,38,1
6463,2018,27,45,1
6464,2018,28,44,1


In [319]:
baseline_df.drop(['year_caught'], axis=1, inplace=True)

In [334]:
baseline_test = sample_sub.copy()

In [329]:
class BaselinePredictor:
    def __init__(self, df):
        self.df = df

    def predict_turtles_rescued_all(self):
        # Initialize an empty list to store dataframes
        dfs = []

        # Get unique capture sites and weeks in the baseline_df
        capture_site_all = self.df['capture_site'].unique()
        weeks_in_year = self.df['week_caught'].unique()

        # Iterate over each capture site and week to calculate the mean turtles_rescued
        for capture_site in capture_site_all:
            for week in weeks_in_year:
                # Calculate mean turtles_rescued for the current capture site and week
                mean_turtles_rescued = self.df[(self.df['capture_site'] == capture_site) & (self.df['week_caught'] == week)]['turtles_rescued'].mean()

                # Append a dataframe to the list
                dfs.append(pd.DataFrame({'capture_site': [capture_site], 'week_caught': [week], 'turtles_rescued': [mean_turtles_rescued]}))

        # Concatenate all dataframes in the list
        predict_df = pd.concat(dfs, ignore_index=True)

        return predict_df

In [330]:
# Initialize the predictor with the baseline_df
predictor = BaselinePredictor(baseline_df)

# Predict the baseline values
predict_baseline = predictor.predict_turtles_rescued_all()

# Print the predicted baseline DataFrame
print(predict_baseline)

      capture_site  week_caught  turtles_rescued
0                2            2         1.666667
1                2            3         1.333333
2                2            4         1.000000
3                2            5         1.500000
4                2            7         1.000000
...            ...          ...              ...
1532            17           37         2.333333
1533            17           39         1.000000
1534            17           33         1.250000
1535            17           20         1.000000
1536            17           53         1.000000

[1537 rows x 3 columns]


In [340]:
from sklearn.metrics import mean_absolute_error

# Ensure both DataFrames have the same number of rows
num_rows_ = baseline_test.shape[0]

# Randomly sample rows from predict_baseline to match the number of rows in sample_sub
predict_baseline_trimmed = predict_baseline.sample(n=num_rows_sample_sub, random_state=42).reset_index(drop=True)

# Ensure the indices match
baseline_test = baseline_test.reset_index(drop=True)

# Combine both DataFrames to ensure we drop NaNs in corresponding rows
combined_df = pd.concat([baseline_test['turtles_rescued'], predict_baseline_trimmed['turtles_rescued']], axis=1, keys=['true', 'pred'])

# Drop rows with NaN values in either column
combined_df = combined_df.dropna()

# Separate the true and predicted values
y_true = combined_df['true']
y_pred = combined_df['pred']

# Calculate MAE
mae_baseline = mean_absolute_error(y_true, y_pred)
print(mae_baseline)


3.2108299243509903


## Contextualising MAE

In [331]:
print(baseline_test.turtles_rescued.min())
print(baseline_test.turtles_rescued.max())

0
9


In [332]:
target_min = 1
target_max = 9
target_range = target_max - target_min
acceptable_mae = target_range * 0.1  # Example threshold of 10% of the range
print(f"Acceptable MAE: {acceptable_mae}")

Acceptable MAE: 0.8


## RMSE Baseline

In [341]:
from sklearn.metrics import mean_squared_error

# Calculate RMSE
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

# Summary statistics
print(sample_sub['turtles_rescued'].describe())

# Baseline RMSE
mean_turtles_rescued = baseline_test['turtles_rescued'].mean()
baseline_predictions = [mean_turtles_rescued] * len(sample_sub)
baseline_mse = mean_squared_error(baseline_test['turtles_rescued'], baseline_predictions)
baseline_rmse = np.sqrt(baseline_mse)
print(f"Baseline RMSE: {baseline_rmse}")

# Coefficient of Variation of RMSE
cv_rmse = (rmse / mean_turtles_rescued) * 100
print(f"Coefficient of Variation of RMSE: {cv_rmse:.2f}%")

# Standard Deviation of Turtles Rescued
std_turtles_rescued = baseline_test['turtles_rescued'].std()
print(f"Standard Deviation of Turtles Rescued: {std_turtles_rescued}")
print(f"RMSE as a proportion of Standard Deviation: {rmse / std_turtles_rescued}")


Root Mean Squared Error: 3.9440045387035063
count    1276.000000
mean        4.436520
std         2.878706
min         0.000000
25%         2.000000
50%         4.000000
75%         7.000000
max         9.000000
Name: turtles_rescued, dtype: float64
Baseline RMSE: 2.8775776437795377
Coefficient of Variation of RMSE: 88.90%
Standard Deviation of Turtles Rescued: 2.878705884420333
RMSE as a proportion of Standard Deviation: 1.370061651677794


## Benchmarking against baseline

In [342]:
# Calculate the MAE for the baseline model
baseline_mae = mean_absolute_error(sample_sub['turtles_rescued'], baseline_predictions)
print(f"Baseline MAE: {baseline_mae}")

# Compare baseline MAE with your model's MAE
if mae_baseline < baseline_mae:
    print("Your model is performing better than the baseline.")
else:
    print("Your model is not performing better than the baseline.")


Baseline MAE: 2.519082703589784
Your model is not performing better than the baseline.


## Interactive version

In [ ]:
def predict_turtles_rescued(capture_site, week):
    predict_df = baseline_df[(baseline_df['capture_site'] == 5) & (baseline_df['week_caught'] == 5)]
    turtle_rescued = predict_df['turtles_rescued'].mean()
    return turtle_rescued

In [ ]:
week = input(f'Enter a week number for which you would like to predict (from 1 to {len(baseline_df.week_caught.unique())}):')
capture_site = input(f'Enter a capture site number (from 1 to 29):')
print(f'Predicted Turtles Rescued for {capture_site} and {week}: {predict_turtles_rescued(capture_site, week)}')

# Linear Regression

In [325]:
import sklearn.linear_model 

In [326]:
linear_regression = linear_model.LinearRegression()

In [327]:
x_train_lm = train_df.groupby(['capture_site', 'week_caught']).size()
x_train_lm = train_df.reset_index(name='rescue_count')


TypeError: DataFrame.reset_index() got an unexpected keyword argument 'name'

In [ ]:
rescue_counts = train_df.groupby(['capture_site', 'week_caught']).size().reset_index(name='rescue_count')
train_df = pd.merge(train_df, rescue_counts, on=['capture_site', 'week_caught'], how='left')

In [ ]:
y = train_df['rescue_count']

In [ ]:
train_df.info()

In [ ]:
X.isna().sum()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming 'X' is your feature matrix and 'y' is your target variable
# Split the data into training and testing sets (optional)
columns_to_drop = ['rescue_id', 'capture_site', 'turtle_characteristics', 'tag_1', 'tag_2', 'lost_tags', 't_number', 'sex', 'status', 'rescue_count', 'foraging_ground', 'capture_method', 'landing_site', 'status', 'release_site', 'week_release', 'year_release', 'weight_kg', 'ccl_cm', 'ccw_cm']

X = train_df.drop(columns=columns_to_drop)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the linear regression model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Optionally, make predictions
y_pred = model.predict(X_test)

# Optionally, evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score

# Compute regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display the evaluation metrics
print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared (R2):", r2)
print("F1 Score:", f1)
